# Rank and Partition

Bu challenge'ın amacı, `RANK` veya `SUM` gibi Window Functions kullanmaktır (bkz. [Window Functions](https://mode.com/sql-tutorial/sql-window-functions/)) ve bunları `OVER` keyword'ü ile uygulamaktır.

`OVER` clause şu şekilde kullanılabilir:
- Satırların kapsamını seçmek için `PARTITION BY`
- Window function'ın uygulanacağı satırların sıralamasını belirlemek için `ORDER BY`


## Data
Bu URL'de bulunan `ecommerce.sqlite` database ile çalışacağız:
`https://wagon-public-datasets.s3.amazonaws.com/sql_databases/ecommerce.sqlite`

Dosyayı indirmek için aşağıdaki hücreyi çalıştırın:

In [1]:
!curl https://wagon-public-datasets.s3.amazonaws.com/sql_databases/ecommerce.sqlite > data/ecommerce.sqlite

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9216  100  9216    0     0  10388      0 --:--:-- --:--:-- --:--:-- 10378


## Database Discovery

Bu database'i daha önce keşfettiniz. Database'i indirdikten sonra, VS Code SQLite extension'ını kullanarak tekrar inceleyebileceğinizi unutmayın.

- `Ctrl-Shift-P` veya `Cmd-Shift-P` tuşlarına basın
- `SQLite: Open Database` görünene kadar yazmaya başlayın
- `Enter`
- Açılan listeden database dosyasını seçin
- Sol alt köşede `SQLITE EXPLORER` butonuna tıklayın


## Setup

Pandas ve sqlite3 ihtiyacımız olan her şey :-)

In [7]:
import pandas as pd
from sqlite3 import connect

In [8]:
tables = pd.read_sql("""
SELECT name 
FROM sqlite_master 
WHERE type='table';
""", conn)

tables


,name
0,Customers
1,Shippers
2,Employees
3,Orders
4,OrderDetails


In [9]:
pd.read_sql("SELECT * FROM orders LIMIT 5", conn)



,OrderID,CustomerID,EmployeeID,OrderDate,RequiredDate,ShippedDate,ShipVia,FreightCharge
0,1,1,1,2012-01-04,2012-01-09,2012-01-05,1,3.75
1,2,2,2,2012-01-27,2012-02-01,2012-01-28,1,7.25
2,3,4,1,2012-02-19,2012-02-24,2012-02-23,2,5.50
3,4,2,4,2012-03-13,2012-03-18,2012-03-14,2,13.50
4,5,4,2,2012-04-05,2012-04-10,2012-04-06,3,8.75


In [11]:
pd.read_sql("SELECT * FROM orderdetails LIMIT 5", conn)



,OrderDetailID,OrderID,ProductID,UnitPrice,Quantity
0,1,1,1,1.10,30
1,2,1,2,0.25,60
2,3,2,3,5.00,80
3,4,2,4,1.39,110
4,5,2,5,9.97,140


## Order rank per customer

👉 Her müşteri için order date'e göre orders kayıtlarını sıralayan bir SQL Query yazın.

Her müşteri için orders kronolojik olarak sıralanmalıdır. `OrderID`, `CustomerID`, `OrderDate`, `OrderRank` döndürün.

In [3]:
# Her müşterinin siparişlerini sipariş tarihine göre sıralayın.
# Her müşteri için siparişler kronolojik sırayla sıralanmalıdır.
# `OrderID`, `CustomerID`, `OrderDate`, `OrderRank` değerlerini return edin.
query_order_rank_per_customer = """


     SELECT OrderID, CustomerID, OrderDate, RANK() OVER( PARTITION BY CustomerID Order by OrderDate) as OrderRank FROM Orders
"""

In [4]:
with connect('data/ecommerce.sqlite') as conn:
    df = pd.read_sql(
        query_order_rank_per_customer,
        con=conn
    )
df.head()

,OrderID,CustomerID,OrderDate,OrderRank
0,1,1,2012-01-04,1
1,8,1,2012-06-13,2
2,12,1,2012-09-13,3
3,2,2,2012-01-27,1
4,4,2,2012-03-13,2


Sonuç beklediğiniz gibi göründüğünde, sorgunuzu denemek ve test etmek için aşağıdaki hücreleri çalıştırın.

In [5]:
from nbresult import ChallengeResult
result = ChallengeResult(
    'order_rank_per_customer',
    query=query_order_rank_per_customer
)
result.write(); print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /Users/senemyilmaz/.pyenv/versions/workintech/bin/python
cachedir: .pytest_cache
rootdir: /Users/senemyilmaz/data-rank-and-partition/tests
plugins: anyio-4.8.0, typeguard-4.4.2
collecting ... collected 1 item

test_order_rank_per_customer.py::TestOrderRankPerCustomer::test_order_rank_per_customer PASSED [100%]

============================== 1 passed in 0.01s ===============================


💯 You can commit your code:

git add tests/order_rank_per_customer.pickle

git commit -m 'Completed order_rank_per_customer step'

git push origin master



## Order cumulative amount per customer

👉 Her müşteri için order date’e göre orders kayıtlarının kümülatif amount (USD cinsinden) değerini hesaplayan bir SQL Query yazın.

Her müşteri için orders kronolojik olarak sıralanmalıdır. `OrderID`, `CustomerID`, `OrderDate`, `OrderCumulativeAmount` döndürün.

In [26]:
# Sipariş tarihine göre her müşterinin siparişlerinin toplam tutarını (USD cinsinden) döndürün.
# Her müşteri için siparişler kronolojik sırayla sıralanmalıdır.
# `OrderID`, `CustomerID`, `OrderDate`, `OrderCumulativeAmount` değerlerini return edin.
query_order_cumulative_amount_per_customer = """
     SELECT o.OrderID, o.CustomerID, o.OrderDate, SUM(SUM(od.Quantity * od.UnitPrice)) 
     OVER ( PARTITION BY o.CustomerID ORDER BY o.OrderDate) as OrderCumulativeAmount 
     FROM Orders as o 
     JOIN OrderDetails as od ON o.OrderID = od.OrderID GROUP BY o.OrderID ORDER BY o.CustomerID
"""

In [22]:
query_order_cumulative_amount_per_customer = """
    SELECT
        Orders.OrderID,
        Orders.CustomerID,
        Orders.OrderDate,
        SUM(SUM(OrderDetails.UnitPrice * OrderDetails.Quantity)) OVER (
            PARTITION BY Orders.CustomerID
            ORDER BY Orders.OrderDate
        ) AS OrderCumulativeAmount
    FROM Orders
    JOIN OrderDetails ON Orders.OrderID = OrderDetails.OrderID
    GROUP BY Orders.OrderID
    ORDER BY Orders.CustomerID
"""

In [27]:
with connect('data/ecommerce.sqlite') as conn:
    df = pd.read_sql(
        query_order_cumulative_amount_per_customer,
        con=conn
    )
df.head()

,OrderID,CustomerID,OrderDate,OrderCumulativeAmount
0,1,1,2012-01-04,48.0
1,8,1,2012-06-13,1989.7
2,12,1,2012-09-13,2021.7
3,2,2,2012-01-27,1948.7
4,4,2,2012-03-13,2348.7


Sonuç beklediğiniz gibi göründüğünde, sorgunuzu denemek ve test etmek için aşağıdaki hücreleri çalıştırın.

In [28]:
from nbresult import ChallengeResult
result = ChallengeResult(
    'order_cumulative_amount_per_customer',
    query=query_order_cumulative_amount_per_customer
)
result.write(); print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /Users/senemyilmaz/.pyenv/versions/workintech/bin/python
cachedir: .pytest_cache
rootdir: /Users/senemyilmaz/data-rank-and-partition/tests
plugins: anyio-4.8.0, typeguard-4.4.2
collecting ... collected 1 item

test_order_cumulative_amount_per_customer.py::TestOrderCumulativeAmountPerCustomer::test_order_cumulative_amount_per_customer PASSED [100%]

============================== 1 passed in 0.02s ===============================


💯 You can commit your code:

git add tests/order_cumulative_amount_per_customer.pickle

git commit -m 'Completed order_cumulative_amount_per_customer step'

git push origin master



## Key learning points

- Window functions, satırları **aggregate etmeden** farklı satırlardaki verileri hesaplamanıza olanak tanır.
- Satırları aggregate etmek istiyorsanız `GROUP BY` kullanın.
- Eğer aggregate etmek istemiyorsanız, bunun yerine `OVER` ile `PARTITION BY` ve `ORDER BY` kullanın.